## [教學目標]
學習 sklearn 中，各種評估指標的使用與意義

## [參考資料]
- [超詳細解說 AUC (英文)](https://www.dataschool.io/roc-curves-and-auc-explained/):深入了解超常用的指標 AUC
- [更多評估指標](https://zhuanlan.zhihu.com/p/30721429):衡量機器學習模型的準確度
- [如何辨別機器學習模型的好壞？秒懂Confusion Matrix](https://www.ycc.idv.tw/confusion-matrix.html)

## [範例重點]
注意觀察各指標的數值範圍，以及輸入函數中的資料格式

In [1]:
from sklearn import metrics, datasets
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
import numpy as np
%matplotlib inline

## 回歸問題
常見的評估指標有
- MAE
- MSE
- R-square

我們隨機生成(X, y)資料，然後使用線性回歸模型做預測，再使用 MAE, MSE, R-square 評估

In [2]:
X, y = datasets.make_regression(n_features=1, random_state=42, noise=4) # 生成資料
model = LinearRegression() # 建立回歸模型
model.fit(X, y) # 將資料放進模型訓練
prediction = model.predict(X) # 進行預測
mae = metrics.mean_absolute_error(prediction, y) # 使用 MAE 評估
mse = metrics.mean_squared_error(prediction, y) # 使用 MSE 評估
r2 = metrics.r2_score(prediction, y) # 使用 r-square 評估
print("MAE: ", mae)
print("MSE: ", mse)
print("R-square: ", r2)

MAE:  2.841797252565566
MSE:  12.48868006739824
R-square:  0.9916581036260311


## 分類問題
常見的評估指標有
- AUC
- F1-Score (Precision, Recall)

In [3]:
cancer = datasets.load_breast_cancer() # 我們使用 sklearn 內含的乳癌資料集
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, test_size=50, random_state=0)

In [4]:
print(y_test) # 測試集中的 label

[0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 0 1 0 1 1 0 1 1 1 0]


In [5]:
y_pred = np.random.random((50,)) # 我們先隨機生成 50 筆預測值，範圍都在 0~1 之間，代表機率值

In [6]:
print(y_pred)

[0.14720172 0.80579195 0.76034856 0.60860904 0.61743443 0.35238111
 0.51933948 0.07343378 0.03664304 0.66501601 0.36054849 0.61237906
 0.07462593 0.67079372 0.74680641 0.1862449  0.23691693 0.1560744
 0.13743504 0.61234118 0.14525223 0.67742425 0.9062186  0.28587883
 0.97403872 0.31358619 0.36223062 0.81380436 0.96056903 0.40472791
 0.17474258 0.01146383 0.50767647 0.47591229 0.19239177 0.92069738
 0.29015594 0.93565125 0.57276018 0.41466439 0.33723919 0.85818851
 0.35321327 0.66528778 0.15927613 0.66615838 0.1557025  0.9498305
 0.83796782 0.00942152]


### AUC

In [7]:
auc = metrics.roc_auc_score(y_test, y_pred) # 使用 roc_auc_score 來評估。 **這邊特別注意 y_pred 必須要放機率值進去!**
print("AUC: ", auc) # 得到結果約 0.5，與亂猜的結果相近，因為我們的預測值是用隨機生成的

AUC:  0.5551782682512734


## F1-Score

In [8]:
threshold = 0.5 
y_pred_binarized = np.where(y_pred>threshold, 1, 0) # 使用 np.where 函數, 將 y_pred > 0.5 的值變為 1，小於 0.5 的為 0
f1 = metrics.f1_score(y_test, y_pred_binarized) # 使用 F1-Score 評估
precision = metrics.precision_score(y_test, y_pred_binarized) # 使用 Precision 評估
recall  = metrics.recall_score(y_test, y_pred_binarized) # 使用 recall 評估
print("F1-Score: ", f1) 
print("Precision: ", precision)
print("Recall: ", recall)
print("Verify F1-Score:", 2*(precision*recall)/(precision+recall))

F1-Score:  0.6181818181818182
Precision:  0.7083333333333334
Recall:  0.5483870967741935
Verify F1-Score: 0.6181818181818182


## [作業重點]
了解 F1-score 的公式意義，並試著理解程式碼

## 作業
請參考 F1-score 的公式與[原始碼](https://github.com/scikit-learn/scikit-learn/blob/bac89c2/sklearn/metrics/classification.py#L620)，試著寫出 F2-Score 的計算函數

## 練習時間
### F1-Score 其實是 F-Score 中的 β 值為 1 的特例，代表 Precision 與 Recall 的權重相同

請參考 F1-score 的[公式](https://en.wikipedia.org/wiki/F1_score) 與下圖的 F2-score 公式圖，試著寫出 F2-Score 的計算函數 (即$\beta=2$)
> $F_{2}$ measure, which weighs recall higher than precision (by placing more emphasis on false negatives), and the$F_{0.5}$ measure, which weighs recall lower than precision (by attenuating the influence of false negatives).  
當$\beta=1$時就是$F_{1}$ Measure，代表Precision和Recall都同等重要，那如果我希望多看中一點Precision，那$\beta$就可以選擇小一點，當$\beta=0$時，F Measure就是Precision；如果我希望多看中一點Recall，那$\beta$就可以選擇大一點，當$\beta$無限大時，F Measure就是Recall。

![image.png](https://wikimedia.org/api/rest_v1/media/math/render/svg/136f45612c08805f4254f63d2f2524bc25075fff)
The formula in terms of Type I and type II errors:
![image.png](https://wikimedia.org/api/rest_v1/media/math/render/svg/6fbeb471033fdd63a2c2ca7830afc7abdf8b8134)

HINT: 可使用 slearn.metrics 中的 precision, recall 函數幫忙

In [9]:
import numpy as np
y_pred = np.random.randint(2, size=100)  # 生成 100 個隨機的 0 / 1 prediction
y_true = np.random.randint(2, size=100)  # 生成 100 個隨機的 0 / 1 ground truth

In [10]:
y_pred

array([0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1,
       1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0])

In [11]:
precision = metrics.precision_score(y_true, y_pred)
recall = metrics.recall_score(y_true, y_pred)
beta = 2
f2_score = (1 + beta**2)*(precision*recall)/( (beta**2)*precision + recall )
print("Precision: ", precision)
print("Recall: ", recall)
print('F2 Score:', f2_score)

Precision:  0.5490196078431373
Recall:  0.56
F2 Score: 0.5577689243027889


In [12]:
#定義F_beta function
def fbeta_score(y_true, y_pred, beta=1):
    precision = metrics.precision_score(y_true, y_pred) # 計算 Precision
    recall = metrics.recall_score(y_true, y_pred) # 計算 Recall
    
    fbeta = (1+ (beta)**2) * (precision*recall) / (((beta)**2*precision) + recall)
    return fbeta 

In [13]:
print('F2 Score:', fbeta_score(y_true, y_pred, beta=2))

F2 Score: 0.5577689243027889
